# Importing the dependencies

In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Data Collection and pre-processing

In [34]:
#Importing the dataset
movies_data = pd.read_csv('movies.csv')

In [35]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
# Number of Rows and Columns in the dataset
movies_data.shape

(4803, 24)

In [5]:
# Statistics of the data
movies_data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [6]:
# Selecting the relevant features for recommendation
selected_feature = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(selected_feature)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [7]:
# replacing the null values with null string

for feature in selected_feature:
    movies_data[feature] = movies_data[feature].fillna('')

In [8]:
# combining all the 5 features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [9]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [10]:
# Converting the text data to feature vectors
vectorizer = TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [12]:
print(feature_vectors)

  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.24713765026964
  (4801, 17266)	0.28860981849329476
  (4801, 13835)	0.27870029291200094
  (4801, 13175)	0.28860981849329476
  (4801, 171

# Cosine Similarity

In [13]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [14]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [15]:
print(similarity.shape)

(4803, 4803)


# Getting the Movie Name from the user

In [16]:
# getting the movie name from the user
movie_name = input("Enter the favourite movie name: ")

Enter the favourite movie name:  America


In [17]:
# creating a list with all the movie names given in dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [18]:
# finding the close match for the moviename given by the year

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Mi America', 'The American', 'American Pie']


In [19]:
close_match = find_close_match[0]
print(close_match)

Mi America


In [20]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

4065


In [21]:
# getting a list of similar movies_
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.0), (1, 0.0), (2, 0.018837243404602726), (3, 0.06980805147346561), (4, 0.0), (5, 0.0), (6, 0.017101147622551306), (7, 0.0468319384827783), (8, 0.023126182739454798), (9, 0.0), (10, 0.0), (11, 0.062020535367436694), (12, 0.0), (13, 0.0), (14, 0.022244487332520997), (15, 0.016131221945514133), (16, 0.06738418250249817), (17, 0.0), (18, 0.02139224441941281), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0031196243258349585), (25, 0.0033293843155676023), (26, 0.030733356660641913), (27, 0.0), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.028992470912699048), (32, 0.0), (33, 0.0), (34, 0.0), (35, 0.02357713820191266), (36, 0.021244883018294546), (37, 0.0), (38, 0.0), (39, 0.0), (40, 0.023723582177339882), (41, 0.0), (42, 0.02237300175411728), (43, 0.0), (44, 0.0), (45, 0.002999742929384314), (46, 0.021523017588914225), (47, 0.0), (48, 0.0), (49, 0.002767152455682926), (50, 0.0), (51, 0.0), (52, 0.020226536266033057), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.0), (5

In [22]:
# Sorting the based on their similarity scores
sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse=True)
print(sorted_similar_movies)

[(4065, 1.0), (3792, 0.18196497323065533), (2605, 0.13616993425319313), (4281, 0.135118619120109), (2009, 0.12544672448824612), (3637, 0.11764696763484803), (615, 0.11738487225372178), (3250, 0.1130350559129282), (1358, 0.10894943417105951), (1781, 0.1077265942989248), (1314, 0.10640985129319905), (713, 0.1060837666822734), (2880, 0.10599778117485525), (3394, 0.10288126536167752), (953, 0.1014975059355974), (1558, 0.10085021719457007), (703, 0.09915518382104067), (2543, 0.09823685703155044), (4313, 0.09820180528356012), (4359, 0.09801920808227621), (1225, 0.09590474770763999), (3333, 0.09575593789147487), (3831, 0.09459097677587745), (1133, 0.09415398127147068), (2086, 0.09340003597286496), (2280, 0.08895612331977666), (4778, 0.08826208385680887), (283, 0.0866317723087351), (1728, 0.08585894609182697), (4154, 0.08421217941236309), (2771, 0.08248729200376195), (3526, 0.08216876116119487), (4033, 0.08174277275723525), (1972, 0.08109009446563728), (4142, 0.08097665897014938), (510, 0.0803

In [23]:
# print the similar movies based on their indexes

print("Movies Suggested for you: \n")

i=1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if (i<30):
        print(i, '.', title_from_index)
        i+=1

Movies Suggested for you: 

1 . Mi America
2 . Adore
3 . Brazil
4 . Rebecca
5 . Coco Before Chanel
6 . Kill the Messenger
7 . Money Train
8 . Much Ado About Nothing
9 . Austin Powers: The Spy Who Shagged Me
10 . A Walk Among the Tombstones
11 . Extreme Measures
12 . The Horse Whisperer
13 . Charlie Bartlett
14 . Freeheld
15 . Gremlins 2: The New Batch
16 . Analyze This
17 . Two Weeks Notice
18 . Cop Land
19 . Below Zero
20 . The Skeleton Twins
21 . Mickey Blue Eyes
22 . The Omega Code
23 . In Cold Blood
24 . 15 Minutes
25 . Home Alone 2: Lost in New York
26 . Sea of Love
27 . Rampage
28 . The Taking of Pelham 1 2 3
29 . We Own the Night


# Movie Recommendation System

In [32]:
movie_name = input("Enter the favourite movie name: ")

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse=True)

print("Movies Suggested for you: \n")

i=1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if (i<30):
        print(i, '.', title_from_index)
        i+=1

Enter the favourite movie name:  Finding Nemo


Movies Suggested for you: 

1 . Finding Nemo
2 . Shark Tale
3 . Big Fish
4 . John Carter
5 . The Muse
6 . Megamind
7 . Mallrats
8 . Spider-Man
9 . Happy Feet
10 . The Last Temptation of Christ
11 . Meet the Deedles
12 . The English Patient
13 . Death Sentence
14 . When Did You Last See Your Father?
15 . The Life Aquatic with Steve Zissou
16 . Flight of the Intruder
17 . El Mariachi
18 . Treading Water
19 . Silent Movie
20 . Bottle Rocket
21 . WALL·E
22 . The Outsiders
23 . He Got Game
24 . Ponyo
25 . American Dreamz
26 . Zoom
27 . Shooting Fish
28 . Bambi
29 . Dear Frankie
